# Pumping test evaluation - Fitting Neumann (Theis) to measured data
This notebook demonstrate the application of the Theis principle for pumping test evaluation in confined, transient setups. The notebook is based on an Spreadsheet from Prof. Rudolf Liedl.

## Introduction

### General situation
We consider a confined aquifer with constant transmissivity. If a well is pumping water out of the aquifer, radial flow towards the well is induced. The following figure illustrates this situation.

<img src="FIGS/Flow_well_confined.png" width="400">

*Fig: Radial flow in a confined aquifer (Fetter, 1994)*

The calculate the hydraulic situation, the following simplified flow equation can be used. This equation accounts for 1D radial transient flow towards a fully penetrating well within a confined aquifer without further sinks and sources:

$$ \frac{\partial^2 h}{\partial r^2}+\frac{1}{r}\frac{\partial h}{\partial r}=\frac{S}{T}\frac{\partial h}{\partial t} $$

### Solution by Theis for confined aquifers
Charles V. Theis presented a solution for this by deriving

$$ s(r,t)=\frac{Q}{4\pi T}W(u) $$

with the well function

$$ W(u) = \int_{u }^{+\infty} \frac{e^{-\tilde u}}{\tilde u}d\tilde u $$

and the dimensionless variable 

$$ u = \frac{Sr^2}{4Tt} $$

This equations are not easy to solve. Historically, values for the well function were provided by tables or as so called type-curve. The type-curve matching with experimental data for pumping test analysis can be considered as one of the basic hydrogeological methods.

However, modern computer provide an easier and more convinient way to solve the 1D radial flow equation based on the Theis approach. Subsequently, the Theis equation is solved with Python routines. The results for the measured data are graphically presented.

#### Curve fitting with s-t plots

With this approach, time and drawdown are provided by the Theis solution with 

$$ s =\frac{Q}{4\pi T}W(u) $$

and 

$$ t = \frac{Sr^2}{4T} \frac{1}{u} $$

Subsequently, time and drawdown are plotted and fitted to measured time and drawdown data.

### Solution by Neuman for unconfined aquifers

_PLEASE ADD SOME EXPLANATION HERE_

In [14]:
# (First, the necessary Python functions are initiated.)
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
from ipywidgets import *

**The following section stores the measured data.**

This are two lists with time (in minutes) and drawdown (in meters). Make sure that those elements match each other (e.g. the first time is associated with the first drawdown value etc.)

**Subsequently, the given data for the processing of measured data are provided.**

This are the given data like
- pumping rate $Q$,
- aquifer thickness $b$,
- distance of the observation (where time and drawdown were measured) to the pumping well $r$.

**These data can be adjusted for individual measurements.**

In [75]:
# Data from SYMPLE exercise
m_time = [1,1.5,2,2.5,3,4,5,6,8,10,12,14,18,24,30,40,50,60,100,120] # time in minutes
m_time_s = [i*60 for i in m_time] # time in seconds
m_ddown = [0.66,0.87,0.99,1.11,1.21,1.36,1.49,1.59,1.75,1.86,1.97,2.08,2.20,2.36,2.49,2.65,2.78,2.88,3.16,3.28]   # drawdown in meters
num_times = len(m_time)

# Parameters needed to solve Theis (From the SYMPLE example/excercise)
r = 120       # m
b = 8.5       # m
Qs = 0.3/60   # m^3/s
Qd = Qs*60*24 # m^3/d

# (Here the necessary functions like the well function $W(u)$ are defined. Later, those functions are used in the computation)
# Define a function, class, and object for Theis Well analysis

def well_function(u):
    return scipy.special.exp1(u)
    
# (Here, the methode computes the data for the well function. Those data can be used to generate a type curve.)
u_min = -5
u_max = 4

u = np.logspace(u_min,u_max)
u_inv = 1/u
u_inv_a = [4.00E-01, 8.00E-01, 1.40E+00, 2.40E+00, 4.00E+00, 8.00E+00, 1.40E+01, 2.40E+01, 4.00E+01, 8.00E+01, 1.40E+02, 2.40E+02, 4.00E+02, 8.00E+02, 1.40E+03, 2.40E+03, 4.00E+03, 8.00E+03]
u_inv_b = [1.40E-02, 2.40E-02, 4.00E-02, 8.00E-02, 1.40E-01, 2.40E-01, 4.00E-01, 8.00E-01, 1.40E+00, 2.40E+00, 4.00E+00, 8.00E+00, 1.40E+01, 2.40E+01, 4.00E+01, 8.00E+01, 1.40E+02, 2.40E+02, 4.00E+02, 8.00E+02, 1.00E+03]


w_u = well_function(u)

# Neuman type curve data from tables

w_u_a = [[2.48E-02, 2.41E-02, 2.30E-02, 2.14E-02, 1.88E-02, 1.70E-02, 1.38E-02, 1.00E-02, 1.00E-02],
         [1.45E-01, 1.40E-01, 1.31E-01, 1.19E-01, 9.88E-02, 8.49E-02, 6.03E-02, 3.17E-02, 1.74E-02],
         [3.58E-01, 3.45E-01, 3.18E-01, 2.79E-01, 2.17E-01, 1.75E-01, 1.07E-01, 4.45E-02, 2.10E-02],
         [6.62E-01, 6.33E-01, 5.70E-01, 4.83E-01, 3.43E-01, 2.56E-01, 1.33E-01, 4.76E-02, 2.14E-02],
         [1.02E+00, 9.63E-01, 8.49E-01, 6.88E-01, 4.38E-01, 3.00E-01, 1.40E-01, 4.78E-02, 2.15E-02],
         [1.57E+00, 1.46E+00, 1.23E+00, 9.18E-01, 4.97E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [2.05E+00, 1.88E+00, 1.51E+00, 1.03E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [2.52E+00, 2.27E+00, 1.73E+00, 1.07E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [2.97E+00, 2.61E+00, 1.85E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [3.56E+00, 3.00E+00, 1.92E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [4.01E+00, 3.23E+00, 1.93E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [4.42E+00, 3.37E+00, 1.94E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [4.77E+00, 3.43E+00, 1.94E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [5.16E+00, 3.45E+00, 1.94E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [5.40E+00, 3.46E+00, 1.94E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [5.54E+00, 3.46E+00, 1.94E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [5.59E+00, 3.46E+00, 1.94E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02],
         [5.62E+00, 3.46E+00, 1.94E+00, 1.08E+00, 5.07E-01, 3.17E-01, 1.41E-01, 4.78E-02, 2.15E-02]]

w_u_b = [[5.62E+00, 3.46E+00, 1.94E+00, 1.09E+00, 5.12E-01, 3.23E-01, 1.45E-01, 5.09E-02, 2.39E-02],
         [5.62E+00, 3.46E+00, 1.94E+00, 1.09E+00, 5.12E-01, 3.23E-01, 1.47E-01, 5.32E-02, 2.57E-02],
         [5.62E+00, 3.46E+00, 1.94E+00, 1.09E+00, 5.16E-01, 3.27E-01, 1.52E-01, 5.68E-02, 2.86E-02],
         [5.62E+00, 3.46E+00, 1.94E+00, 1.09E+00, 5.24E-01, 3.37E-01, 1.62E-01, 6.61E-02, 3.62E-02],
         [5.62E+00, 3.46E+00, 1.94E+00, 1.10E+00, 5.37E-01, 3.50E-01, 1.78E-01, 8.06E-02, 4.86E-02],
         [5.62E+00, 3.46E+00, 1.95E+00, 1.11E+00, 5.57E-01, 3.74E-01, 2.05E-01, 1.06E-01, 7.14E-02],
         [5.62E+00, 3.46E+00, 1.96E+00, 1.13E+00, 5.89E-01, 4.12E-01, 2.48E-01, 1.49E-01, 1.13E-01],
         [5.62E+00, 3.46E+00, 1.98E+00, 1.18E+00, 6.67E-01, 5.06E-01, 3.57E-01, 2.66E-01, 2.31E-01],
         [5.63E+00, 3.47E+00, 2.01E+00, 1.24E+00, 7.80E-01, 6.42E-01, 5.17E-01, 4.45E-01, 4.19E-01],
         [5.63E+00, 3.49E+00, 2.06E+00, 1.35E+00, 9.54E-01, 8.50E-01, 7.63E-01, 7.18E-01, 7.03E-01],
         [5.63E+00, 3.51E+00, 2.13E+00, 1.50E+00, 1.20E+00, 1.13E+00, 1.08E+00, 1.06E+00, 1.05E+00],
         [5.64E+00, 3.56E+00, 2.31E+00, 1.85E+00, 1.68E+00, 1.65E+00, 1.63E+00, 9.99E+02, 9.99E+02],
         [5.65E+00, 3.63E+00, 2.55E+00, 2.23E+00, 2.15E+00, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02],
         [5.67E+00, 3.74E+00, 2.86E+00, 2.68E+00, 2.65E+00, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02],
         [5.70E+00, 3.90E+00, 3.24E+00, 3.15E+00, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02],
         [5.76E+00, 4.22E+00, 3.85E+00, 3.82E+00, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02],
         [5.85E+00, 4.58E+00, 4.38E+00, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02],
         [5.99E+00, 5.00E+00, 4.91E+00, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02],
         [6.16E+00, 5.46E+00, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02],
         [6.47E+00, 6.11E+00, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02],
         [6.60E+00, 6.50E+00, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02, 9.99E+02]]

t_a_NEU = [0]*len(u_inv_a)
s_a_NEU = [0]*len(u_inv_a)
t_b_NEU = [0]*len(u_inv_b)
s_b_NEU = [0]*len(u_inv_b)

# This is the function to plot the graph with the data, it is called by interact, arguments need to be provided by sliders etc.
def Theis_plot(T,b,Ss, SY, eta):
    # Compute K and SS to provide parameters for plausability check
    # (i.e. are the parameter in a reasonable range)
    K = T/b     # m/s
    S = Ss * b

    # Early (a) and late (b) Theis curve
    t_a_term = r**2 * S / 4 / T
    t_b_term = r**2 * SY / 4 / T
    s_term = Qs/(4 * np.pi * T)

    t_a = u_inv * t_a_term
    t_b = u_inv * t_b_term
    s = w_u * s_term

    # Early Neuman curve
    for x in range(0,len(u_inv_a)):
        t_a_NEU[x] = u_inv_a[x] * t_a_term
        s_a_NEU[x] = w_u_a[x][eta-1] * s_term
    
    # Late Neuman curve
    for x in range(0,len(u_inv_b)):
        t_b_NEU[x] = u_inv_b[x] * t_b_term
        if (w_u_b[x][eta-1] == 999):
            s_b_NEU[x] = well_function(1/u_inv_b[x]) * s_term
        else:
            s_b_NEU[x] = w_u_b[x][eta-1] * s_term
        
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(t_a, s, label=r'Computed drawdown early -Theis')
    ax.plot(t_b, s, label=r'Computed drawdown late -Theis')
    ax.plot(t_a_NEU, s_a_NEU, 'b--', label=r'Computed drawdown early - Neuman')
    ax.plot(t_b_NEU, s_b_NEU, '--', color='darkorange', label=r'Computed drawdown late - Neuman')
    ax.plot(m_time_s, m_ddown,'ro', label=r'measured drawdown')
    plt.yscale("log")
    plt.xscale("log")
    plt.axis([1E-1,1E8,1E-4,1E+1])
    ax.set(xlabel='t', ylabel='s',title='Neuman drawdown')
    ax.grid(which="both")
    plt.legend()
    plt.show()

    print("Transmissivity       T = ","% 10.2E"% T, " m^2/s")
    print("Storativity          S = ","% 10.2E"% S, "[-]")

**The pumping test evaluation is more or less the calibration of a model (i.e. the Theis equations) by adjusting parameters. Those parameters include**
- **Transmissivity** of the entire aquifer $T$
- **Storativity** of the aquifer $S$.

See what happens and modify the parameters $T$ and $S$. Modify until the fit is fine.

_(Hint: After selecting the slider you can use the mouse, your fingers, but also the arrow-keys of your keyboard to modify the parameter values.)_

In [76]:
interact(Theis_plot,
         T=widgets.FloatLogSlider(value=0.001,base=10,min=-6, max=3, step=0.1,readout=True,readout_format='.2e'),
         b = widgets.FloatSlider(value=20,min=2, max=200, step=0.1,readout=True,readout_format='.2f'),
         Ss = widgets.FloatLogSlider(value=0.0001,base=10,min=-9, max=1, step=0.1,readout=True,readout_format='.2e'),
         SY = widgets.FloatSlider(value=0.2,min=0.01, max=0.5, step=0.01,readout=True,readout_format='.2f'),
         eta = widgets.IntSlider(value=1, min = 1, max = 9, step=1, readout=True))

interactive(children=(FloatLogSlider(value=0.001, description='T', max=3.0, min=-6.0, readout_format='.2e'), F…

<function __main__.Theis_plot(T, b, Ss, SY, eta)>